## Looney Tunes Database 

### Data loading

In [126]:
%load -s s3://neptune-demo-s3workingbucket-12zo6uwzrgc4k/looney_tunes.ttl -f turtle --named-graph-uri=http://looneytunes-graph.com

Button(description='Submit', style=ButtonStyle())

Output()

### First query

In [127]:
%%sparql

# Most basic command
# Bring anything that meets hte pattern subject-predicate-object (From the Looney Tunes graph)
SELECT ?s ?p ?o
FROM <http://looneytunes-graph.com>
WHERE {
    ?s ?p ?o
} LIMIT 100

### One-hop away

In [19]:
%%sparql

# What is Bugs Bunny's name?
PREFIX : <http://looneytunes-graph.com/>

SELECT ?n
FROM <http://looneytunes-graph.com>
WHERE {
    :Bugs_Bunny :name ?n
}

In [20]:
%%sparql

# Who is Bugs Bunny's creator?
PREFIX : <http://looneytunes-graph.com/>

SELECT ?c
FROM <http://looneytunes-graph.com>
WHERE {
    :Bugs_Bunny :created_by ?c
}

In [23]:
%%sparql

# What are Bugs Bunny;s Personality Traits?
PREFIX : <http://looneytunes-graph.com/>

SELECT ?t
FROM <http://looneytunes-graph.com>
WHERE {
    :Sylvester :personality_trait ?t
}

### Two-hop away query pattern

In [24]:
%%sparql

# What is/are the name(s) of the creator(s) of Bugs Bunny?
PREFIX : <http://looneytunes-graph.com/>

SELECT ?n
FROM <http://looneytunes-graph.com>
WHERE {
    :Bugs_Bunny :created_by ?p .   # the '.' means 'AND'
    ?p :name ?n
}

In [25]:
%%sparql

# What was the release date when Bugs Bunny made his debut appearance?
PREFIX : <http://looneytunes-graph.com/>

SELECT ?d
FROM <http://looneytunes-graph.com>
WHERE {
    :Bugs_Bunny :made_debut_appearance_in ?n .   # the '.' means 'AND'
    ?n :release_date ?d
}

### Writing Lean queries

In [29]:
%%sparql

# What was the release date when Bugs Bunny made his debut appearance?
PREFIX : <http://looneytunes-graph.com/>

SELECT ?n ?d
FROM <http://looneytunes-graph.com>
WHERE {
    ?c a :Looney_Tunes_Character ;    # Semicolomn allows you to add triplet patterns with the same subject,  predicate 'a' is widely recognised and prefix not needed
     :name ?n ;
     :made_debut_appearance_in ?m .
    ?m :release_date ?d
}

### Counting things in the graph

In Sparql counting is an aggregation function

In [31]:
%%sparql

# How many characters were created by Tex Avery?
PREFIX : <http://looneytunes-graph.com/>

SELECT (COUNT (?c) AS ?cCount)
FROM <http://looneytunes-graph.com>
WHERE {
    ?c :created_by :Tex_Avery
}

### Removing Duplicates from results

In [33]:
%%sparql

# How many types of movies have our Looney Tunes characters made debut appearances in?
PREFIX : <http://looneytunes-graph.com/>

SELECT (COUNT (DISTINCT ?mt) AS ?mtC)
FROM <http://looneytunes-graph.com>
WHERE {
    ?c :made_debut_appearance_in ?m .
    ?m a ?mt
}

### Checking the existance or non-existance of certain patterns

In [37]:
%%sparql

# Find Looney Tunes characters that have a catchphrase
PREFIX : <http://looneytunes-graph.com/>

SELECT ?c
FROM <http://looneytunes-graph.com>
WHERE {
    ?c a :Looney_Tunes_Character .
    
    FILTER NOT EXISTS {
        ?c :known_for_catchphrase ?p
    }
}

In [41]:
%%sparql

# Who are the characters who were co-created?
PREFIX : <http://looneytunes-graph.com/>

SELECT DISTINCT ?c
FROM <http://looneytunes-graph.com>
WHERE {
    ?c :created_by ?p1 , ?p2 .
    
    FILTER(?p1 != ?p2) 
}

### MIN and MAX functions

In [42]:
%%sparql

# What are the earliest and latest dates for movies?
PREFIX : <http://looneytunes-graph.com/>

SELECT (MIN(?date) AS ?earliest) (MAX(?date) AS ?latest)
FROM <http://looneytunes-graph.com>
WHERE {
    ?m a :Short ;
        :release_date ?date
}

### Bind variables (simple mathematical operation)

In [44]:
%%sparql

# How old was Tex Avery when he died?
PREFIX : <http://looneytunes-graph.com/>

SELECT ?age
FROM <http://looneytunes-graph.com>
WHERE {
    :Tex_Avery :born_on ?b ;
               :died_on ?d .
    
    BIND (year(?b) AS ?bYear)
    BIND (year(?d) AS ?dYear)
    BIND ((?dYear - ?bYear) AS ?age)
}

In [46]:
%%sparql

# Lifespan of each Looney Tunes character creator
PREFIX : <http://looneytunes-graph.com/>

SELECT ?c ?age
FROM <http://looneytunes-graph.com>
WHERE {
    ?c :born_on ?b ;
               :died_on ?d .
    
    BIND (year(?b) AS ?bYear)
    BIND (year(?d) AS ?dYear)
    BIND ((?dYear - ?bYear) AS ?age)
}

In [48]:
%%sparql

# How was the longest living looney tunes creator?
PREFIX : <http://looneytunes-graph.com/>

SELECT ?c ?age
FROM <http://looneytunes-graph.com>
WHERE {
    ?c :born_on ?b ;
               :died_on ?d .
    
    BIND (year(?b) AS ?bYear)
    BIND (year(?d) AS ?dYear)
    BIND ((?dYear - ?bYear) AS ?age)
}
ORDER BY DESC (?age)
LIMIT 1

### Computing average values

In [51]:
%%sparql

# What was the average death age of all Looney Tunes character creators combined?
PREFIX : <http://looneytunes-graph.com/>

SELECT (AVG(?age) AS ?avgAge)
FROM <http://looneytunes-graph.com>
WHERE {
    ?c :born_on ?b ;
               :died_on ?d .
    
    BIND (year(?b) AS ?bYear)
    BIND (year(?d) AS ?dYear)
    BIND ((?dYear - ?bYear) AS ?age)
}
ORDER BY DESC (?age)
LIMIT 1

### Running tests, More advanced use of BIND, COALESCE and IF

In [56]:
%%sparql

# What if we wanted to say something like:
# If release date between 1946 and 1950 => post-war period release
# If release date between 1939 and 1945 => WW2 release
# If release date between 1918 and 1938 => Interwar period
PREFIX : <http://looneytunes-graph.com/>

SELECT ?n ?result
FROM <http://looneytunes-graph.com>
WHERE {
    ?c :made_debut_appearance_in ?m ;
       :name ?n .
    ?m :release_date ?d .
    
    BIND (year(?d) AS ?dYear) .
    
    BIND (
        IF(?dYear >= 1946 && ?dYear <=1950, "Released during post-war era",
           IF(?dYear >= 1939 && ?dYear <=1945, "Released during WW2",
              IF(?dYear >= 1918 && ?dYear <=1938, "Released during interwar period", "Other era"
                )
             )
          ) AS ?result
    )
}
ORDER BY ASC(?n)

In [57]:
%%sparql

# What if we wanted to say something like:
# If release date between 1946 and 1950 => post-war period release
# If release date between 1939 and 1945 => WW2 release
# If release date between 1918 and 1938 => Interwar period
PREFIX : <http://looneytunes-graph.com/>

SELECT ?n ?result
FROM <http://looneytunes-graph.com>
WHERE {
    ?c :made_debut_appearance_in ?m ;
       :name ?n .
    ?m :release_date ?d .
    
    BIND (year(?d) AS ?dYear) .
    
    BIND (
    COALESCE(
        IF(?dYear >= 1946 && ?dYear <=1950, "Released during post-war era", 1/0),
        IF(?dYear >= 1939 && ?dYear <=1945, "Released during WW2", 1/0),
        IF(?dYear >= 1918 && ?dYear <=1938, "Released during interwar period", 1/0), 
        "Other era"
        ) AS ?result
    )
}
ORDER BY ASC(?n)

### Working with OPTIONAL matches

In [59]:
%%sparql

# What are the catchphrases of all the different characters (include characters that may bnot have catchphrases)?
PREFIX : <http://looneytunes-graph.com/>

SELECT ?c ?cp
FROM <http://looneytunes-graph.com>
WHERE {
    ?c a :Looney_Tunes_Character .
    
    OPTIONAL {
        ?c :known_for_catchphrase ?cp
    }
}

### The UNION clause

In [61]:
%%sparql

# What is the full list of all the names of things in our dataset (i.e., names of Looney Tunes characters and their creators)
PREFIX : <http://looneytunes-graph.com/>

SELECT ?n
FROM <http://looneytunes-graph.com>
WHERE {
    {
      ?c a :Looney_Tunes_Character ;
           :name ?n 
    }
    UNION
    {
      ?c a :Person ;
           :name ?n 
    }
}
ORDER BY ASC(?n)

### Excluding results with the MINUS clause

In [85]:
%%sparql

# Remove Tasmanian Devil from the list of names
PREFIX : <http://looneytunes-graph.com/>

SELECT ?n
FROM <http://looneytunes-graph.com>
WHERE {
    {
      ?c a :Looney_Tunes_Character ;
           :name ?n 
    }
    MINUS
    {
      ?c :name "Tasmanian Devil"
    }
}
ORDER BY ASC(?n)

### GROUPING

In [65]:
%%sparql

# What is the count of recorded personality traits for each of our Looney Tunes characters?
PREFIX : <http://looneytunes-graph.com/>

SELECT ?c (COUNT(?pt) AS ?ptCount)
FROM <http://looneytunes-graph.com>
WHERE {
    ?c :personality_trait ?pt
}
GROUP BY ?c 

### DESCRIBE query

In [68]:
%%sparql

# What are incoming and outgoing predicates to and from Bugs Bunny?
PREFIX : <http://looneytunes-graph.com/>

DESCRIBE :Bugs_Bunny  # Describes an entity giving you a picture of what is around it a single hop away

### ASK query

In [71]:
%%sparql

# Is it true that Bugs Bunny was created by Tex Avery?
PREFIX : <http://looneytunes-graph.com/>

ASK {
    :Bugs_Bunny :created_by :Tex_Avery
}

### What is CONSTRUCT code?
A way of creating a subgraph

In [72]:
%%sparql

# Return an RDF sub-graph which contains specific triple patters - persons and their brith and death dates
PREFIX : <http://looneytunes-graph.com/>

CONSTRUCT
WHERE {
    ?p a :Person ;
         :born_on ?b ;
         :died_on ?d .
}

In [73]:
%%sparql

# Return an RDF sub-graph which contains specific triple patters - persons and their average age
PREFIX : <http://looneytunes-graph.com/>

CONSTRUCT{
    ?p :has_age ?age
}
WHERE {
    ?p a :Person ;
         :born_on ?b ;
         :died_on ?d .
    
    BIND(year(?b) AS ?bYear)
    BIND(year(?d) AS ?dYear)
    BIND((?dYear - ?bYear) AS ?age)
}

### Property Paths
Traverse multiple hops at scale and efficiently

### Inverse paths

In [75]:
%%sparql

# Inverse path
PREFIX : <http://looneytunes-graph.com/>

SELECT ?c ?p
FROM <http://looneytunes-graph.com>
WHERE {
    ?p ^:created_by ?c  # The hat says to look for the inverse direction
}

In [76]:
%%sparql

# Sequence paths - list characters who are enemies of another who themselves are rivals of other characters 
# (I have not added these edges to the graph so results will be empty)
PREFIX : <http://looneytunes-graph.com/>

SELECT ?c1 ?c2
FROM <http://looneytunes-graph.com>
WHERE {
    ?c1 :enemy_of/:rival_of ?c2
}

In [77]:
%%sparql

# Recursive paths - who knows Taz (directly or transitively) through one or more occurrences of "knows"?
# (I have not added these edges to the graph so results will be empty)
PREFIX : <http://looneytunes-graph.com/>

SELECT ?c
FROM <http://looneytunes-graph.com>
WHERE {
    ?c :knows+ :Taz  # WildCard to say that anythings that is connected (arbitrary length) or + sign to say one or more hops away
}

### Listing possible path

In [86]:
%%sparql

# What are the possible paths I can take between two nodes (e.g. between Bugs Bunny and Daffy Duck)?
# (I have not added these edges to the graph so results will be empty)

PREFIX : <http://looneytunes-graph.com/>

SELECT DISTINCT ?subject ?predicate ?object
FROM <http://looneytunes-graph.com>
WHERE {
    VALUES ?start { :Sylvester }
    VALUES ?end { :Taz }
    
    ?start (:|!:)* ?subject .  # (:|!:)* a predicate that exists in the graph, zero or more times (doesn't consider inverse paths)
    ?subject ?predicate ?object . 
    ?object (:|!:)* ?end
}

### CREATE, UPDATE and DELETE data

In [109]:
%%sparql

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX : <http://looneytunes-graph.com/>


INSERT DATA {
    
# Bugs Bunny
:Bugs_Bunny a :Looney_Tunes_Character ;
  :name "Bugs Bunny" ;
  :species "Hare" ;
  :gender "Male" ;
  :made_debut_appearance_in :A_Wild_Hare ;
  :created_by :Tex_Avery ;
  :personality_trait "Cunning" , "Charismatic" , "Smart" ;
  :known_for_catchphrase "What's up, doc?" .

# A Wild Hare
:A_Wild_Hare a :Short ;
  :release_date "1940-07-27"^^xsd:date .

# Tex Avery
:Tex_Avery a :Person ;
  :name "Frederick Bean Avery" ; 
  :born_on "1908-02-26"^^xsd:date ;
  :died_on "1980-08-26"^^xsd:date .

# Daffy Duck
:Daffy_Duck a :Looney_Tunes_Character ;
  :name "Daffy Damas Duck" ;
  :species "American Black Duck" ;
  :gender "Male" ;
  :made_debut_appearance_in :Porkys_Duck_Hunt ;
  :created_by :Tex_Avery ;
  :personality_trait "Assertive" , "Unrestrained" , "Combative" ;
  :known_for_catchphrase "You're despicable!" .

# Porky's Duck Hunt
:Porkys_Duck_Hunt a :Short ;
  :release_date "1937-04-17"^^xsd:date .

# Sylvester 
:Sylvester a :Looney_Tunes_Character ;
  :name "Sylvester James Pussycat, Sr." ;
  :species "Tuxedo Cat" ;
  :gender "Male" ;
  :made_debut_appearance_in :Life_With_Feathers ;
  :created_by :Friz_Freleng ;
  :personality_trait "Proud" , "Persistent" , "Loser" ;
  :known_for_catchphrase "Sufferin' Succotash" .

# Life With Feathers
:Life_With_Feathers a :Short ;
  :release_date "1945-03-24"^^xsd:date .

# Friz Frelang
:Friz_Freleng a :Person ;
  :name "Isadore Freleng" ;
  :born_on "1905-08-21"^^xsd:date ;
  :died_on "1995-05-26"^^xsd:date .

# Taz
:Taz a :Looney_Tunes_Character ;
  :name "Tasmanian Devil" ;
  :gender "Male" ;
  :species "Tasmanian Devil" ;
  :made_debut_appearance_in :Devil_May_Hare ;
  :created_by :Robert_McKimson , :Sid_Marcus ;
  :personality_trait "Dim-witted" , "Short-tempered" , "Impatient" .

# Devil May Hare
:Devil_May_Hare a :Short ;
  :release_date "1954-06-19"^^xsd:date .

# Robert McKimson
:Robert_McKimson a :Person ;
  :name "Robert Porter McKimson, Sr." ;
  :born_on "1910-10-13"^^xsd:date ;
  :died_on "1977-09-29"^^xsd:date .

# Sid Marcus
:Sid_Marcus a :Person ;
  :name "Sidney Marcus" ;
  :born_on "1877-10-14"^^xsd:date ;
  :died_on "1979-01-31"^^xsd:date .
}

### Deleting specific triplets

In [102]:
%%sparql
# What are the possible paths I can take between two nodes (e.g. between Bugs Bunny and Daffy Duck)?
# (I have not added these edges to the graph so results will be empty)

PREFIX : <http://looneytunes-graph.com/>

DELETE DATA {
    # Bugs Bunny
:Bugs_Bunny a :Looney_Tunes_Character ;
  :name "Bugs Bunny" ;
  :species "Hare" ;
  :gender "Male" ;
  :made_debut_appearance_in :A_Wild_Hare ;
  :created_by :Tex_Avery ;
  :personality_trait "Cunning" , "Charismatic" , "Smart" ;
  :known_for_catchphrase "What's up, doc?" .

# A Wild Hare
:A_Wild_Hare a :Short ;
  :release_date "1940-07-27"^^xsd:date .

# Tex Avery
:Tex_Avery a :Person ;
  :name "Frederick Bean Avery" ; 
  :born_on "1908-02-26"^^xsd:date ;
  :died_on "1980-08-26"^^xsd:date .

# Daffy Duck
:Daffy_Duck a :Looney_Tunes_Character ;
  :name "Daffy Damas Duck" ;
  :species "American Black Duck" ;
  :gender "Male" ;
  :made_debut_appearance_in :Porkys_Duck_Hunt ;
  :created_by :Tex_Avery ;
  :personality_trait "Assertive" , "Unrestrained" , "Combative" ;
  :known_for_catchphrase "You're despicable!" .

# Porky's Duck Hunt
:Porkys_Duck_Hunt a :Short ;
  :release_date "1937-04-17"^^xsd:date .

# Sylvester 
:Sylvester a :Looney_Tunes_Character ;
  :name "Sylvester James Pussycat, Sr." ;
  :species "Tuxedo Cat" ;
  :gender "Male" ;
  :made_debut_appearance_in :Life_With_Feathers ;
  :created_by :Friz_Freleng ;
  :personality_trait "Proud" , "Persistent" , "Loser" ;
  :known_for_catchphrase "Sufferin' Succotash" .

# Life With Feathers
:Life_With_Feathers a :Short ;
  :release_date "1945-03-24"^^xsd:date .

# Friz Frelang
:Friz_Freleng a :Person ;
  :name "Isadore Freleng" ;
  :born_on "1905-08-21"^^xsd:date ;
  :died_on "1995-05-26"^^xsd:date .

# Taz
:Taz a :Looney_Tunes_Character ;
  :name "Tasmanian Devil" ;
  :gender "Male" ;
  :species "Tasmanian Devil" ;
  :made_debut_appearance_in :Devil_May_Hare ;
  :created_by :Robert_McKimson , :Sid_Marcus ;
  :personality_trait "Dim-witted" , "Short-tempered" , "Impatient" .

# Devil May Hare
:Devil_May_Hare a :Short ;
  :release_date "1954-06-19"^^xsd:date .

# Robert McKimson
:Robert_McKimson a :Person ;
  :name "Robert Porter McKimson, Sr." ;
  :born_on "1910-10-13"^^xsd:date ;
  :died_on "1977-09-29"^^xsd:date .

# Sid Marcus
:Sid_Marcus a :Person ;
  :name "Sidney Marcus" ;
  :born_on "1877-10-14"^^xsd:date ;
  :died_on "1979-01-31"^^xsd:date .
}

In [124]:
%%sparql

# Most basic command
# Bring anything that meets hte pattern subject-predicate-object (From the Looney Tunes graph)
SELECT ?s ?p ?o
FROM <http://looneytunes-graph.com>
WHERE {
    ?s ?p ?o
} LIMIT 100

In [115]:
%%sparql

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX : <http://looneytunes-graph.com/>


INSERT {
    ?c2 :knows ?c1
}
WHERE {
    ?c1 :knows ?c2
}

In [116]:
%%sparql

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX : <http://looneytunes-graph.com/>


DELETE WHERE {?s ?p ?o}


### Named graphs

#### Drop graph

In [123]:
%%sparql

DROP GRAPH <http://looneytunes-graph.com>

In [125]:
%%sparql

SELECT ?g 
WHERE {
  GRAPH ?g { }
}

### Understanding the Schema or structure the graph

In [128]:
%%sparql

# Understand the types int he graph
SELECT DISTINCT ?t
FROM <http://looneytunes-graph.com>
WHERE {
    ?s a ?t
}
GROUP BY ?t
ORDER BY ASC(?t)

In [131]:
%%sparql

# Understand the predicates in the graph
SELECT DISTINCT ?p
FROM <http://looneytunes-graph.com>
WHERE {
    ?s ?p ?o
}
ORDER BY ASC(?p)

In [132]:
%%sparql

# What is the full vocabulary for the graph?
SELECT DISTINCT ?t ?p
FROM <http://looneytunes-graph.com>
WHERE {
    ?s ?p ?o ;
        a ?t
}
GROUP BY ?t ?p
ORDER BY ASC(?t) ASC(?p)

#### Distinguish literals from types

In [135]:
%%sparql

# What is the full vocabulary for the graph?
SELECT DISTINCT ?t ?p ?pType
FROM <http://looneytunes-graph.com>
WHERE {
    ?s ?p ?o ;
        a ?t
    BIND (
        COALESCE(
            IF(isIRI(?o), "Resource Predicate", 1/0),
            "Literal Predicate"
            ) AS ?pType
        )
}
GROUP BY ?t ?p ?pType
ORDER BY ASC(?t) ASC(?p)